In [2]:
import pandas as pd
import numpy as np
import pickle as pkl

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
# download dfs from local source
accidentDF = pd.read_csv('2015-traffic-fatalities/accident.csv', delimiter = ',')
vehicleDF = pd.read_excel('2015-traffic-fatalities/vehicleDF.xls')

In [5]:
# merge dfs - each contain individual case number which can be merged on
df = pd.merge(accidentDF, vehicleDF, on='ST_CASE')

In [6]:
df.head()

,STATE_x,ST_CASE,VE_TOTAL,VE_FORMS_x,PVH_INVL,PEDS,PERNOTMVIT,PERMVIT,PERSONS,COUNTY,CITY,DAY_x,MONTH_x,YEAR,DAY_WEEK,HOUR_x,MINUTE_x,NHS,RUR_URB,FUNC_SYS,RD_OWNER,ROUTE,TWAY_ID,TWAY_ID2,MILEPT,LATITUDE,LONGITUD,SP_JUR,HARM_EV_x,MAN_COLL_x,RELJCT1,RELJCT2,TYP_INT,WRK_ZONE,REL_ROAD,LGT_COND,WEATHER1,WEATHER2,WEATHER,SCH_BUS,RAIL,NOT_HOUR,NOT_MIN,ARR_HOUR,ARR_MIN,HOSP_HR,HOSP_MN,CF1,CF2,CF3,FATALS,DRUNK_DR,STATE_y,VEH_NO,VE_FORMS_y,NUMOCCS,DAY_y,MONTH_y,HOUR_y,MINUTE_y,HARM_EV_y,MAN_COLL_y,UNITTYPE,HIT_RUN,REG_STAT,OWNER,MAKE,MODEL,MAK_MOD,BODY_TYP,MOD_YEAR,VIN,VIN_1,VIN_2,VIN_3,VIN_4,VIN_5,VIN_6,VIN_7,VIN_8,VIN_9,VIN_10,VIN_11,VIN_12,TOW_VEH,J_KNIFE,MCARR_I1,MCARR_I2,MCARR_ID,GVWR,V_CONFIG,CARGO_BT,HAZ_INV,HAZ_PLAC,HAZ_ID,HAZ_CNO,HAZ_REL,BUS_USE,SPEC_USE,EMER_USE,TRAV_SP,UNDERIDE,ROLLOVER,ROLINLOC,IMPACT1,DEFORMED,TOWED,M_HARM,VEH_SC1,VEH_SC2,FIRE_EXP,DR_PRES,L_STATE,DR_ZIP,L_STATUS,L_TYPE,CDL_STAT,L_ENDORS,L_COMPL,L_RESTRI,DR_HGT,DR_WGT,PREV_ACC,PREV_SUS,PREV_DWI,PREV_SPD,PREV_OTH,FIRST_MO,FIRST_YR,LAST_MO,LAST_YR,SPEEDREL,DR_SF1,DR_SF2,DR_SF3,DR_SF4,VTRAFWAY,VNUM_LAN,VSPD_LIM,VALIGN,VPROFILE,VPAVETYP,VSURCOND,VTRAFCON,VTCONT_F,P_CRASH1,P_CRASH2,P_CRASH3,PCRASH4,PCRASH5,ACC_TYPE,DEATHS,DR_DRINK
0,1,10001,1,1,0,0,0,1,1,127,0,1,1,2015,5,2,40,0,1,3,1,3,SR-5,NaN,1754,33.878653,-87.325328,0,35,0,0,1,1,0,4,2,1,0,1,0,0000000,99,99,2,58,88,88,0,0,0,1,1,1,1,1,1,1,1,2,40,35,0,1,0,1,1,12,481,12481,31,2003,1FTRX18L83NB,1,F,T,R,X,1,8,L,8,3,N,B,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,55,0,0,0,12,6,2,42,0,0,0,1,1,35578,6,1,0,0,3,0,73,205,1,0,0,1,0,6,2011,6,2012,0,0,0,0,0,1,2,55,3,1,2,1,0,0,14,13,99,1,4,1,1,1
1,1,10002,1,1,0,0,0,1,1,83,0,1,1,2015,5,22,13,1,1,1,1,1,I-65,NaN,3604,34.910442,-86.908708,0,34,0,0,1,1,0,3,2,10,0,10,0,0000000,99,99,22,20,88,88,0,0,0,1,0,1,1,1,1,1,1,22,13,34,0,1,0,1,1,49,40,49040,4,2006,4T1BE30K66U1,4,T,1,B,E,3,0,K,6,6,U,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,70,0,1,3,6,6,2,1,0,0,0,1,1,60565,6,1,6,0,3,0,70,190,2,2,0,0,0,8,2010,12,2013,4,89,0,0,0,3,2,70,3,1,2,2,0,0,14,6,1,4,4,7,1,0
2,1,10003,1,1,0,0,0,2,2,11,0,1,1,2015,5,1,25,0,1,3,1,2,US-SR 6,NaN,1958,32.142006,-85.758456,0,42,0,0,1,1,0,4,2,1,0,1,0,0000000,99,99,1,45,99,99,0,0,0,1,1,1,1,1,2,1,1,1,25,42,0,1,0,1,1,20,37,20037,4,2008,1G1ZK57728F1,1,G,1,Z,K,5,7,7,2,8,F,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,80,0,1,5,12,6,2,42,0,0,1,1,1,36089,6,1,0,0,3,0,68,157,2,0,0,1,0,8,2010,7,2014,0,0,0,0,0,1,2,55,3,1,2,1,0,0,1,13,1,1,4,1,1,1
3,1,10004,1,1,0,0,0,1,1,45,0,4,1,2015,1,0,57,0,1,4,1,3,SR-27,NaN,566,31.439814,-85.510300,0,53,0,0,1,1,0,4,2,10,0,10,0,0000000,99,99,1,15,88,88,0,0,0,1,1,1,1,1,1,4,1,0,57,53,0,1,0,12,2,12,481,12481,31,2005,1FTRF12245KE,1,F,T,R,F,1,2,2,4,5,K,E,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,75,0,0,0,12,6,2,42,0,0,0,1,12,36360,6,1,0,0,3,0,72,999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,55,2,1,2,1,0,0,14,13,1,1,4,1,1,1
4,1,10005,2,2,0,0,0,2,2,45,2050,7,1,2015,4,7,9,0,2,3,1,2,US-SR 53,HINTON WATERS AVE,308,31.319331,-85.515100,0,12,6,0,2,3,0,1,1,1,0,1,0,0000000,99,99,7,16,88,88,0,0,0,1,0,1,1,2,1,7,1,7,9,12,6,1,0,1,1,69,54,69054,3,2006,WMWRC33536TK,W,M,W,R,C,3,3,5,3,6,T,K,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,15,0,1,5,3,6,2,12,0,0,0,1,1,36079,6,1,0,0,3,1,73,230,0,0,0,0,0,0,0,0,0,0,38,0,0,0,2,2,65,1,6,2,1,0,0,11,15,1,1,1,68,1,0


In [41]:
# pick features I think can be indicative of target
df = df[['DR_DRINK', 'HOUR_x', 'VE_FORMS_x', 'VE_TOTAL', 'PERSONS', 'PEDS', 'NHS', 'HIT_RUN', 'ROLLOVER', 'TRAV_SP', 'VSPD_LIM', 'PREV_ACC', 'PREV_SUS', 'PREV_SPD', 'PREV_OTH']]
df.head()

,DR_DRINK,HOUR_x,VE_FORMS_x,VE_TOTAL,PERSONS,PEDS,NHS,HIT_RUN,ROLLOVER,TRAV_SP,VSPD_LIM,PREV_ACC,PREV_SUS,PREV_SPD,PREV_OTH
0,1,2,1,1,1,0,0,0,0,55,55,1,0,1,0
1,0,22,1,1,1,0,1,0,1,70,70,2,2,0,0
2,1,1,1,1,2,0,0,0,1,80,55,2,0,1,0
3,1,0,1,1,1,0,0,0,0,75,55,0,0,0,0
4,0,7,2,2,2,0,0,0,1,15,65,0,0,0,0


In [42]:
# clean up data (remove numbers representing unknowns)
df = df[(df.NHS != 9)]
df = df[(df.HIT_RUN != 9)]
df = df[(df.ROLLOVER != 9) & (df.ROLLOVER != 2)]
df = df[(df.TRAV_SP != 0) & (df.TRAV_SP < 151)]
df = df[(df.VSPD_LIM != 0) & (df.TRAV_SP < 98)]
df = df[(df.PREV_ACC < 10)]

In [43]:
# create new dummy variable for if the crash occurs between the hours of 7 PM and 7 AM and
# if the crash is a single vehicle crash
df['BETWEEN_7PM_AND_7AM'] = np.where((df['HOUR_x'] < 7) | (df['HOUR_x'] > 18), 1, 0)
df['SINGLE_VEHICLE_CRASH'] = np.where(df['VE_TOTAL'] == 1, 1, 0)

In [44]:
# final edits to remove unnecessary columns and rename others
df.rename(columns = {'VE_FORMS_x':'IN_TRANSPORT_VEHICLES_INVOLVED', 'VE_TOTAL':'TOTAL_VEHICLES_INVOLVED', 'NHS':'ON_HIGHWAY', 'DR_DRINK':'DRUNK_DRIVER'}, inplace=True)
df.drop(['HOUR_x'], axis=1, inplace=True)

In [45]:
# create new feature and rename feature
df['SPEED_ABOVE_SPEED_LIMIT'] = df["TRAV_SP"] - df['VSPD_LIM']
df.rename(columns = {'HOUR_x':'HOUR'}, inplace=True)

In [46]:
# create new column (string for drunk_or_sober dummy variable)
df.rename(columns = {'DRUNK_DRIVER':'DRUNK_OR_SOBER'}, inplace=True)
df['DRUNK_DRIVER'] = np.where(df['DRUNK_OR_SOBER'] == 1, 'Drunk', 'Sober')

In [49]:
# select features to be used
df = df[['DRUNK_DRIVER', 'BETWEEN_7PM_AND_7AM', 'ROLLOVER', 'SINGLE_VEHICLE_CRASH', 'PEDS', 'SPEED_ABOVE_SPEED_LIMIT']]

In [50]:
df.head()

,DRUNK_DRIVER,BETWEEN_7PM_AND_7AM,ROLLOVER,SINGLE_VEHICLE_CRASH,PEDS,SPEED_ABOVE_SPEED_LIMIT
0,Drunk,1,0,1,0,0
1,Sober,1,1,1,0,0
2,Drunk,1,1,1,0,25
3,Drunk,1,0,1,0,20
4,Sober,0,1,0,0,-50


In [51]:
# view differences between feature means for drunk/sober
df.groupby(['DRUNK_DRIVER']).mean()

,BETWEEN_7PM_AND_7AM,ROLLOVER,SINGLE_VEHICLE_CRASH,PEDS,SPEED_ABOVE_SPEED_LIMIT
DRUNK_DRIVER,,,,,
Drunk,0.712436,0.287564,0.592389,0.065516,9.389565
Sober,0.381722,0.116291,0.290331,0.176954,-4.494481


In [ ]:
# put df in excel format for tableau to create visuals
df.to_excel('drunkDrivers.xlsx')

In [52]:
# dump df as picklefile
with open('FatalAccidentsEdited', 'wb') as picklefile:
    pkl.dump(df, picklefile)